In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")

# 슬랙 봇 토큰
SLACK_BOT_TOKEN = SLACK_BOT_TOKEN

In [2]:
from slack_sdk import WebClient

# Slack 클라이언트 초기화
client = WebClient(token=SLACK_BOT_TOKEN)  # 직접 토큰 입력 (환경 변수 대신)

def send_message(message: str, channel: str):
    try:
        client.chat_postMessage(channel=channel, text=message)
        print("메시지가 전송되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")

if __name__ == "__main__":
    send_message("안녕하세요, Slack 테스트 메시지입니다!", CHANNEL_NAME)


메시지가 전송되었습니다.


In [3]:
import os
from dotenv import load_dotenv
import python_bithumb

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")

In [ ]:
access_key = BITHUMB_API_KEY 
secret_key = BITHUMB_SECRET_KEY  

bithumb = python_bithumb.Bithumb(access_key, secret_key)

krw_balance = bithumb.get_balance("KRW")

send_message(
    f"""
        안녕하세요 오전 8시마다 보내드리는 알림입니다.
    ------------------------------------
    전체 자산 : {krw_balance}원

    전체 수익률 : 

    전날 대비 수익률 : 
    ------------------------------------        
    활기찬 연구생활 되시기를 바랍니다.
    """, CHANNEL_NAME
)

메시지가 전송되었습니다.
